In [7]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.applications.vgg16 import VGG16, decode_predictions, preprocess_input
from keras.optimizers import SGD
from keras.preprocessing import image
import numpy as np

# Dimensions of our images.
img_width, img_height = 150, 150  #lowering computations

train_data_dir = '/content/drive/MyDrive/training'
validation_data_dir = '/content/drive/MyDrive/Validation'
test_data_dir = '/content/drive/MyDrive/test'
nb_train_samples = 177 # 60
nb_validation_samples = 114 # 40
epochs = 5
batch_size = 32

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

# Load pretrained VGG16 model.
# The last (top) layers doing the final classification are not included.
vgg16 = VGG16(weights='imagenet', input_shape=input_shape, include_top=False)

# Freeze the weights for the first layers.
for layer in vgg16.layers[:17]:
    layer.trainable = False
# vgg16.summary()

# Add custom layers.
x = vgg16.output
x = Flatten()(x)
x = Dense(units=64, activation="relu")(x)
x = Dropout(0.5)(x)
predictions = Dense(units=3, activation="softmax")(x)
print(vgg16.input)
# Create final model.
#model = Model(inputs = vgg16.input, output = predictions)
model = Model(inputs=vgg16.input, outputs=predictions)
model.compile(loss='CategoricalCrossentropy',
              optimizer=SGD(lr=0.001, momentum=0.9),
              metrics=['accuracy'])
history = model.summary()

tf.keras.utils.plot_model(
  model, to_file='/content/model.png', show_shapes=True, show_dtype=False,
  show_layer_names=True, rankdir='TB', expand_nested=False, dpi=96
  )

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')


validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

print(test_generator)
model.fit(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)



In [50]:
img = image.load_img('/content/drive/MyDrive/test/Fair Personality/9.jpg', target_size=(150, 150))
labels = ['Averge personality', 'fair personality', 'good personality']
array = image.img_to_array(img)
array = np.expand_dims(array, axis=0)
# print(array)
preds = model.predict(preprocess_input(array))
# res = decode_predictions(preds, top=1)[0]

array_preds = max(preds)
(array_preds)
index_pred = np.argmax(array_preds)
labels[index_pred]
# for arr,labels in test_generator:
#   # arr.shape
#   # print(arr)
#   # print(labels)
#   # print/(labels)
#   # plt.imshow(arr)
#   # print('****')
#   # print(arr)
#   res = model.predict(arr)
#   # print(res)learning_rate
#   #print('****')


# # res = model.predict(image_test)
# # print(res)



'fair personality'